In [226]:
import random
import re
from urllib.parse import urlparse, parse_qs, parse_qsl, quote

import pandas as pd
import requests

from tqdm import tqdm

from bs4 import BeautifulSoup
import bs4

URL_ROOT = 'https://en.wiktionary.org'

soups = {}
def get_soup(path):
    if path in soups:
        return soups[path]
    
    res = requests.get(f"{URL_ROOT}{path}")
    soup = BeautifulSoup(res.text)
    soups[path] = soup
    return soup

# e.g. get_category_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_category_pages(start_path):
    paths = [start_path]    
    next_path = start_path
    soup = get_soup(next_path)
    next_link = soup.find('a', string='next page')
    while next_link:
        next_path = next_link['href']
        paths.append(next_path)
        soup = get_soup(next_path)
        next_link = soup.find('a', string='next page')
    return paths

# e.g. get_lemma_pages("/wiki/Category:South_Levantine_Arabic_terms_with_usage_examples")
def get_lemma_pages(path):
    soup = get_soup(path)
    return [{'href': a['href'], 'title': a['title']} for a in soup.css.select('div#mw-pages li a')]

In [227]:
pd.set_option('display.max_colwidth', 250)

In [228]:
# Category:Armenian_terms_with_usage_examples
# Category:Old_Armenian_terms_with_usage_examples

lps = ([lp for cp in get_category_pages('/wiki/Category:Armenian_terms_with_usage_examples')
    for lp in get_lemma_pages(cp)])
lemmas = pd.DataFrame(lps)
lemmas

,href,title
0,/wiki/%D5%A1,ա
1,/wiki/%D5%A1%CC%88,ա̈
2,/wiki/%D5%A1%CC%8A,ա̊
3,/wiki/%D5%A1%D5%A2%D5%A5%D5%A9%D5%A1%D5%BD%D5%B8%D6%82%D5%B6%D5%AF,աբեթասունկ
4,/wiki/%D5%A1%D5%A3%D5%BC%D5%A1%D5%BE,ագռավ
...,...,...
1789,/wiki/%D5%96%D6%80%D5%B8%D6%82%D5%B6%D5%A6%D5%A5,Ֆրունզե
1790,/wiki/%D5%9B,՛
1791,/wiki/%D5%9E,՞
1792,/wiki/%D6%89,։


In [229]:
def getLanguageSections(soup):
    headings = soup.select('div#bodyContent div.mw-heading2')
    
    heading_map = {}

    for heading in headings:
        language = heading.find('h2').text
        content = []
        for sib in heading.next_siblings:
            if sib == '\n':
                continue
            if type(sib) == bs4.element.Comment:
                continue
            if sib in headings:
                break
            content.append(sib)
        heading_map[language] = content
    return heading_map

In [230]:
def getHeadingLevel(cs):
    levels = [c for c in cs if c != 'mw-heading']
    if len(levels) > 0:
        return levels[0]

In [231]:
def getSubSections(section):
    results = []
    result = {}
    current_header = None
    current_subsection = bs4.Tag(name='div')
    top_heading_level = None
    for tag in section:
        classes = tag.attrs.get('class', [])
        if tag.name == 'div' and 'mw-heading' in classes:
            heading_level = getHeadingLevel(classes)
            if not top_heading_level:
                top_heading_level = heading_level
            elif heading_level == top_heading_level and result:
                if len(result) > 1:
                    results.append(result)
                    result = {}
                else:
                    top_heading_level = '_' # non-existent heading
            if current_header:
                result[current_header] = current_subsection
            current_header = tag.text.replace('[edit]', '')
            current_subsection = bs4.Tag(name='div')
        else:
            current_subsection.append(tag)
    if current_header:
        result[current_header] = current_subsection
    results.append(result)
    return results

In [232]:
len(bs4.Tag(name='div'))

0

In [233]:
entries = {}

for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas.sample(100).iterrows()]):
#for title, href in tqdm([row[['title','href']].to_list() for _, row in lemmas[lemmas.title.isin(['աման', 'փիս', '-անք', 'համ'])].iterrows()]):
    sections = getLanguageSections(get_soup(href))
    if 'Armenian' not in sections:
        continue
    entries[title] = getSubSections(sections['Armenian'])

100%|██████████| 100/100 [01:03<00:00,  1.56it/s]


In [234]:
pd.DataFrame([dict(title=k, idx=i+1, **r) for k,v in entries.items() for i,r in enumerate(v)]).set_index(['title', 'idx'])

,,Etymology,Pronunciation,Adjective,Declension,References,Alternative forms,Noun,Derived terms,Adverb,Verb,...,Postposition,Letter,See also,Etymology 3,Antonyms,Conjunction,Conjugation,Particle,Pronoun,Descendants
title,idx,,,,,,,,,,,,,,,,,,,,,
ալեհերձ,1,"[[[<a href=""/wiki/%D5%A1%D5%AC%D5%AB%D6%84#Armenian"" title=""ալիք"">ալիք</a>], , [(], [alikʻ], [)], +‎ , [<a href=""/wiki/-%D5%A1-#Armenian"" title=""-ա-"">-ա-</a>], , [(], [-a-], [)], +‎ , [<a href=""/wiki/%D5%B0%D5%A5%D6%80%D5%B1%D5%A5%D5%AC#Armen...","[[[], \n, [<span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a></span><span class=""ib-brac"">...","[[[<strong class=""Armn headword"" lang=""hy"">ալեհերձ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">aleherj</span>, ...","[[\n, [<small>nominalized, <i>i</i>-type</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;t...","[[[<span class=""citation-whole""><span class=""cited-source"">“<a class=""external text"" href=""http://www.nayiri.com/imagedDictionaryBrowser.jsp?dictionaryId=29&amp;query=%D5%A1%D5%AC%D5%A5%D5%B0%D5%A5%D6%80%D5%B1"" rel=""nofollow""><span class=""Armn"" l...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
բուդ,1,"[[From , [<a class=""extiw"" href=""https://en.wikipedia.org/wiki/Ottoman_Turkish"" title=""w:Ottoman Turkish"">Ottoman Turkish</a>], , [<a href=""/wiki/%D8%A8%D9%88%D8%AF#Ottoman_Turkish"" title=""بود"">بود</a>], , [(], [bud], [)], , , [<a href=""/wiki/%...","[[[<style data-mw-deduplicate=""TemplateStyles:r50165410"">.mw-parser-output .k-player .k-attribution{visibility:hidden}</style>, <table class=""audiotable"" style=""vertical-align: middle; display: inline-block; list-style: none; line-height: 1em; bo...",NaN,"[[\n, [<small> <i>i</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;te...","[[[\n, <li id=""cite_note-1""><a href=""#cite_ref-1""> ^ </a> <span class=""reference-text""><span class=""citation-whole""><span class=""cited-source""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Hrachia_Adjarian"" title=""w:Hrachia Adjarian"">Ačaṙe...","[[[<span class=""Armn"" lang=""hy""><a class=""new"" href=""/w/index.php?title=%D5%BA%D5%B8%D6%82%D5%BF&amp;action=edit&amp;redlink=1"" title=""պուտ (page does not exist)"">պուտ</a></span>, , <span class=""mention-gloss-paren annotation-paren"">(</span>, <s...","[[[<strong class=""Armn headword"" lang=""hy"">բուդ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">bud</span>, )], \n]...","[[[<span class=""Armn"" lang=""hy""><a class=""new"" href=""/w/index.php?title=%D5%A2%D5%A4%D5%A5%D5%AC&amp;action=edit&amp;redlink=1"" title=""բդել (page does not exist)"">բդել</a></span>, , <span class=""mention-gloss-paren annotation-paren"">(</span>, <s...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ինչյան,1,"[[From , [<a href=""/wiki/%D5%AB%D5%B6%D5%B9#Armenian"" title=""ինչ"">ինչ</a>], , [(], [inčʻ], , , [“], [what], [”], [)], +‎ , [<a href=""/wiki/-%D5%B5%D5%A1%D5%B6#Armenian"" title=""-յան"">-յան</a>], , [(], [-yan], , , [surname suffix], [)], .\n]]","[[[<span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern A

In [235]:
results = pd.DataFrame([dict(title=k, idx=i+1, **r) for k,v in entries.items() for i,r in enumerate(v)]).set_index(['title', 'idx'])

In [236]:
results

,,Etymology,Pronunciation,Adjective,Declension,References,Alternative forms,Noun,Derived terms,Adverb,Verb,...,Postposition,Letter,See also,Etymology 3,Antonyms,Conjunction,Conjugation,Particle,Pronoun,Descendants
title,idx,,,,,,,,,,,,,,,,,,,,,
ալեհերձ,1,"[[[<a href=""/wiki/%D5%A1%D5%AC%D5%AB%D6%84#Armenian"" title=""ալիք"">ալիք</a>], , [(], [alikʻ], [)], +‎ , [<a href=""/wiki/-%D5%A1-#Armenian"" title=""-ա-"">-ա-</a>], , [(], [-a-], [)], +‎ , [<a href=""/wiki/%D5%B0%D5%A5%D6%80%D5%B1%D5%A5%D5%AC#Armen...","[[[], \n, [<span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a></span><span class=""ib-brac"">...","[[[<strong class=""Armn headword"" lang=""hy"">ալեհերձ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">aleherj</span>, ...","[[\n, [<small>nominalized, <i>i</i>-type</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;t...","[[[<span class=""citation-whole""><span class=""cited-source"">“<a class=""external text"" href=""http://www.nayiri.com/imagedDictionaryBrowser.jsp?dictionaryId=29&amp;query=%D5%A1%D5%AC%D5%A5%D5%B0%D5%A5%D6%80%D5%B1"" rel=""nofollow""><span class=""Armn"" l...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
բուդ,1,"[[From , [<a class=""extiw"" href=""https://en.wikipedia.org/wiki/Ottoman_Turkish"" title=""w:Ottoman Turkish"">Ottoman Turkish</a>], , [<a href=""/wiki/%D8%A8%D9%88%D8%AF#Ottoman_Turkish"" title=""بود"">بود</a>], , [(], [bud], [)], , , [<a href=""/wiki/%...","[[[<style data-mw-deduplicate=""TemplateStyles:r50165410"">.mw-parser-output .k-player .k-attribution{visibility:hidden}</style>, <table class=""audiotable"" style=""vertical-align: middle; display: inline-block; list-style: none; line-height: 1em; bo...",NaN,"[[\n, [<small> <i>i</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;te...","[[[\n, <li id=""cite_note-1""><a href=""#cite_ref-1""> ^ </a> <span class=""reference-text""><span class=""citation-whole""><span class=""cited-source""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Hrachia_Adjarian"" title=""w:Hrachia Adjarian"">Ačaṙe...","[[[<span class=""Armn"" lang=""hy""><a class=""new"" href=""/w/index.php?title=%D5%BA%D5%B8%D6%82%D5%BF&amp;action=edit&amp;redlink=1"" title=""պուտ (page does not exist)"">պուտ</a></span>, , <span class=""mention-gloss-paren annotation-paren"">(</span>, <s...","[[[<strong class=""Armn headword"" lang=""hy"">բուդ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">bud</span>, )], \n]...","[[[<span class=""Armn"" lang=""hy""><a class=""new"" href=""/w/index.php?title=%D5%A2%D5%A4%D5%A5%D5%AC&amp;action=edit&amp;redlink=1"" title=""բդել (page does not exist)"">բդել</a></span>, , <span class=""mention-gloss-paren annotation-paren"">(</span>, <s...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ինչյան,1,"[[From , [<a href=""/wiki/%D5%AB%D5%B6%D5%B9#Armenian"" title=""ինչ"">ինչ</a>], , [(], [inčʻ], , , [“], [what], [”], [)], +‎ , [<a href=""/wiki/-%D5%B5%D5%A1%D5%B6#Armenian"" title=""-յան"">-յան</a>], , [(], [-yan], , , [surname suffix], [)], .\n]]","[[[<span class=""usage-label-accent""><span class=""ib-brac"">(</span><span class=""ib-content""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern A

In [237]:
columns_by_freq = results.count().sort_values(ascending=False)
results = results[columns_by_freq.index] #[lambda x:x['Verb'].notna()]
columns_by_freq.to_frame().style.bar()

,0
Pronunciation,101
Etymology,91
Declension,81
Noun,53
Adjective,34
Alternative forms,31
Derived terms,29
Related terms,20
Verb,16
References,15


In [238]:
def process_pronunciation(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return '\n'.join([li.text for li in tag.select('li') if not 'Audio' in li.text and not li.text == ''])

results['Pronunciation'] = results['Pronunciation'].apply(process_pronunciation)

In [239]:
def process_basic_text(tag):
    if type(tag) != bs4.Tag:
        return tag
    
    return tag.text.strip()


results['Etymology'] = results['Etymology'].apply(process_basic_text)
results['Etymology 1'] = results['Etymology 1'].apply(process_basic_text)
results['Etymology 2'] = results['Etymology 2'].apply(process_basic_text)

In [240]:
results

,,Pronunciation,Etymology,Declension,Noun,Adjective,Alternative forms,Derived terms,Related terms,Verb,References,...,Antonyms,Descendants,Postposition,Conjunction,Conjugation,Pronoun,Proper noun,Letter,Etymology 3,Particle
title,idx,,,,,,,,,,,,,,,,,,,,,
ալեհերձ,1,"(Eastern Armenian) IPA(key): /ɑleˈheɾd͡z/, [ɑlehéɾd͡z]\n(Western Armenian) IPA(key): /ɑleˈheɾt͡s/, [ɑlehéɾt͡sʰ]",ալիք (alikʻ) +‎ -ա- (-a-) +‎ հերձել (herjel),"[[\n, [<small>nominalized, <i>i</i>-type</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;t...",NaN,"[[[<strong class=""Armn headword"" lang=""hy"">ալեհերձ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">aleherj</span>, ...",NaN,NaN,NaN,NaN,"[[[<span class=""citation-whole""><span class=""cited-source"">“<a class=""external text"" href=""http://www.nayiri.com/imagedDictionaryBrowser.jsp?dictionaryId=29&amp;query=%D5%A1%D5%AC%D5%A5%D5%B0%D5%A5%D6%80%D5%B1"" rel=""nofollow""><span class=""Armn"" l...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
բուդ,1,"(Eastern Armenian) IPA(key): /bud/, [bud]","From Ottoman Turkish بود (bud), بوت (but).[1]","[[\n, [<small> <i>i</i>-type, inanimate</small>, (, <a class=""extiw"" href=""https://en.wikipedia.org/wiki/Eastern_Armenian"" title=""w:Eastern Armenian"">Eastern Armenian</a>, )], \n, [\n, <table class=""inflection-table"" style=""background:#F9F9F9;te...","[[[<strong class=""Armn headword"" lang=""hy"">բուդ</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">bud</span>, )], \n]...",NaN,"[[[<span class=""Armn"" lang=""hy""><a class=""new"" href=""/w/index.php?title=%D5%BA%D5%B8%D6%82%D5%BF&amp;action=edit&amp;redlink=1"" title=""պուտ (page does not exist)"">պուտ</a></span>, , <span class=""mention-gloss-paren annotation-paren"">(</span>, <s...","[[[<span class=""Armn"" lang=""hy""><a class=""new"" href=""/w/index.php?title=%D5%A2%D5%A4%D5%A5%D5%AC&amp;action=edit&amp;redlink=1"" title=""բդել (page does not exist)"">բդել</a></span>, , <span class=""mention-gloss-paren annotation-paren"">(</span>, <s...",NaN,NaN,"[[[\n, <li id=""cite_note-1""><a href=""#cite_ref-1""> ^ </a> <span class=""reference-text""><span class=""citation-whole""><span class=""cited-source""><a class=""extiw"" href=""https://en.wikipedia.org/wiki/Hrachia_Adjarian"" title=""w:Hrachia Adjarian"">Ačaṙe...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ինչյան,1,"(Eastern Armenian) IPA(key): /inˈt͡ʃʰjɑn/, [int͡ʃʰjɑ́n]\n(Western Armenian) IPA(key): /inˈt͡ʃjɑn/, [int͡ʃʰjɑ́n]","From ինչ (inčʻ, “what”) +‎ -յան (-yan, surname suffix).",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ուղարկել,1,"(Eastern Armenian) IPA(key): /uʁɑɾˈkel/, [uʁɑɾkél]\n(Western Armenian) IPA(key): /uʁɑɾˈɡel/, [uʁɑɾɡél]","From Old Armenian ուղարկեմ (ułarkem).\n\n\n\n\n\nThis etymology is missing or incomplete. Please add to it, or discuss it at the Etymology scriptorium. Particularly: “By surface analysis, ուղի (uġi) +‎ արկել (arkel).”",NaN,NaN,NaN,"[[[<span class=""Armn"" lang=""hy""><a href=""/wiki/%D5%B2%D6%80%D5%AF%D5%A5%D5%AC#Armenian"" title=""ղրկել"">ղրկել</a></span>, , <span class=""mention-gloss-paren annotation-paren"">(</span>, <span class=""tr Latn"" lang=""hy-Latn"">ġrkel</span>, <span class...",NaN,NaN,"[[[<strong class=""Armn headword"" lang=""hy"">ուղարկել</strong>, , <a href=""/wiki/Wiktionary:Armenian_transliteration"" title=""Wiktionary:Armenian transliteration"">•</a>, (, <span class=""headword-tr tr Latn"" dir=""ltr"" lang=""hy-Latn"">uġarkel</span>,...","[[[\n, <li id=""cite_note-Orengo-1""><a href=""#cite_ref-Orengo_1-0""> ^ </a> <span class=""reference-text""